In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df = pd.read_csv("../data/total_production_v1.csv")
df.head()

,Crop year,Unnamed: 1,1990/91,1991/92,1992/93,1993/94,1994/95,1995/96,1996/97,1997/98,...,2010/11,2011/12,2012/13,2013/14,2014/15,2015/16,2016/17,2017/18,2018/19,2019/20
0,April group,NaN,40795,42530,46868,40167,39990,29396,44298,39476,...,71811,64395,75896,74494,69873,71332,75508,70324,81708,76076
1,Angola,(R/A),50,79,78,33,77,62,71,64,...,35,29,33,35,39,41,45,35,42,52
2,Bolivia (Plurinational State of),(A),123,104,120,51,117,142,125,141,...,117,132,105,120,100,84,78,84,83,81
3,Brazil,(A/R),27286,27293,34603,28167,28192,18060,29197,26148,...,55428,48592,55418,54689,53305,52871,56788,52740,65131,58211
4,Burundi,(A/R),487,667,620,393,664,434,401,250,...,353,204,406,163,248,269,196,202,204,272


In [4]:
# Remove unnecessary columns
df = df.drop(columns="Unnamed: 1")
# Rename columns: rename, crop year to country
df = df.rename(columns={"Crop year": "country"})

In [5]:
# Remove 'group' from country column value, create list of tuples with country column and edge country that contained group <--> contains index of group edge e.g April group, July group, October group
countries = df["country"]
groups = [(i, countries[i].replace(" group", "")) for i, s in enumerate(countries) if 'group' in s]
groups

[(0, 'April'), (15, 'July'), (23, 'October')]

In [6]:
# Slices df for countries based on interval that signifies their group. Returns a new df to concat with global output df
def slice_df(df, groups, index, start, end):
  group_df = pd.DataFrame(df.loc[start: end-1])
  group_df["group"] = groups[index][1]
  return group_df

In [7]:
# NOTE: Can be improved and needs graphic explaining algorithm
# Second to last index
second_to_last = len(groups) - 1

# Start: start index of group list, End: end index of group list
start, end = None, None

new_df = pd.DataFrame()
for index in range(0, second_to_last):
  start = groups[index][0] + 1
  end = groups[index+1][0]
  new_df = pd.concat([new_df, slice_df(df, groups, index, start, end)])

# Process last group
start = end + 1
end = len(df) - 1
new_df = pd.concat([new_df, slice_df(df, groups, second_to_last, start, end)])


In [8]:
new_df.head()

,country,1990/91,1991/92,1992/93,1993/94,1994/95,1995/96,1996/97,1997/98,1998/99,...,2011/12,2012/13,2013/14,2014/15,2015/16,2016/17,2017/18,2018/19,2019/20,group
1,Angola,50,79,78,33,77,62,71,64,85,...,29,33,35,39,41,45,35,42,52,April
2,Bolivia (Plurinational State of),123,104,120,51,117,142,125,141,138,...,132,105,120,100,84,78,84,83,81,April
3,Brazil,27286,27293,34603,28167,28192,18060,29197,26148,36761,...,48592,55418,54689,53305,52871,56788,52740,65131,58211,April
4,Burundi,487,667,620,393,664,434,401,250,492,...,204,406,163,248,269,196,202,204,272,April
5,Ecuador,1504,2124,1185,2069,2376,1888,1993,1191,1206,...,825,828,666,644,644,645,624,496,559,April


In [9]:
# Standardize country column
new_df["country"] = new_df["country"].map(lambda x: x.split(" (")[0])
new_df["country"] = new_df["country"].str.lower()

In [10]:
# Turn year columns into values e.g 1990/91, create new column named "year" that holds column values
df = new_df.melt(id_vars=["country", "group"], var_name="year", value_name="units")
df.head()

,country,group,year,units
0,angola,April,1990/91,50
1,bolivia,April,1990/91,123
2,brazil,April,1990/91,27286
3,burundi,April,1990/91,487
4,ecuador,April,1990/91,1504


In [11]:
# Reformat year to current year, remove last 3 characters from year string
df["year"] = df["year"].str[:-3]

In [12]:
# EXPORT CSV FILE
# df.to_csv("../data/total_production_v2.csv", index=False)
# print("Done")